In [10]:
!pip install flask
!pip install pyngrok
!pip install pdfplumber
!pip install matplotlib

In [13]:
from flask import Flask, request, render_template_string, send_from_directory
from pyngrok import ngrok
import pdfplumber
import matplotlib.pyplot as plt
import os
import re
import uuid

# Set your ngrok token
ngrok.set_auth_token("2wFrDOFZB3Bx4ZhG8OUGT07NHYY_3ZQsnQcyZAz1tox4Targd")

app = Flask(__name__)
app.config['UPLOAD_FOLDER'] = 'uploads'
app.config['STATIC_FOLDER'] = 'static'
os.makedirs(app.config['UPLOAD_FOLDER'], exist_ok=True)
os.makedirs(app.config['STATIC_FOLDER'], exist_ok=True)

extracted_text = ""  # Global to hold uploaded file text

# --- PDF Extraction ---
def extract_text_from_pdf(file_path):
    text = ""
    with pdfplumber.open(file_path) as pdf:
        for page in pdf.pages:
            page_text = page.extract_text()
            if page_text:
                text += page_text + "\n"
    return text

# --- Chart Generation ---
def generate_revenue_chart(text):
    pattern = r"Total Revenue\s*(\d{4})\s*[:\-]?\s*\$([\d,\.]+)"
    matches = re.findall(pattern, text, re.IGNORECASE)

    if not matches:
        return None

    # Clean and sort
    years = [int(year) for year, val in matches]
    values = [float(val.replace(",", "")) for year, val in matches]
    data = sorted(zip(years, values))
    years, values = zip(*data)

    # Plot
    plt.figure(figsize=(6, 4))
    plt.bar(years, values, color="#3498db")
    plt.title("Total Revenue Over Years")
    plt.xlabel("Year")
    plt.ylabel("Revenue (in billions)")
    plt.tight_layout()

    chart_id = f"{uuid.uuid4()}.png"
    chart_path = os.path.join(app.config['STATIC_FOLDER'], chart_id)
    plt.savefig(chart_path)
    plt.close()

    return f"/static/{chart_id}"

# --- Chatbot Logic ---
def simple_chatbot(user_query):
    query = user_query.lower().strip()

    if "total revenue" in query:
        return ("Here are the total revenues for 2023:<br>"
                "- Microsoft: $232 billion<br>"
                "- Tesla: $97.7 billion<br>"
                "- Apple: $383 billion")
    elif "net income" in query:
        return ("Net Income changes from 2022 to 2023:<br>"
                "- Microsoft: Increased by 0.5%<br>"
                "- Tesla: Increased by 11.8%<br>"
                "- Apple: Decreased by 2.8%")
    elif "highest total assets" in query:
        return "Apple has the highest total assets with $384 billion as of 2023."
    elif "cash flow from operations" in query:
        return "Microsoft generated the most cash flow from operations in 2023, totaling $89 billion."
    elif "highest revenue growth" in query:
        return "Tesla had the highest revenue growth (~18.8%) from 2022 to 2023."

    elif "chart" in query or "visual" in query:
        if not extracted_text:
            return "❌ No file uploaded to generate chart."
        chart_url = generate_revenue_chart(extracted_text)
        if chart_url:
            return f'<img src="{chart_url}" alt="Revenue Chart" style="max-width:100%;">'
        else:
            return "📉 No revenue data found in the uploaded PDF to generate a chart."

    elif "uploaded" in query or "file" in query or "statement" in query:
        if not extracted_text:
            return "No file uploaded yet. Please upload a financial statement PDF."
        rev_match = re.search(r"Total Revenue.*?\$([\d,\.]+)", extracted_text, re.IGNORECASE)
        net_match = re.search(r"Net Income.*?\$([\d,\.]+)", extracted_text, re.IGNORECASE)
        return f"""📄 Based on the uploaded document:<br>
                  • Total Revenue: ${rev_match.group(1) if rev_match else 'Not found'}<br>
                  • Net Income: ${net_match.group(1) if net_match else 'Not found'}"""

    else:
        return "❌ Sorry, I can only provide information on predefined queries or uploaded documents."

# --- Home UI ---
@app.route("/", methods=["GET", "POST"])
def home():
    global extracted_text
    response = ""
    if request.method == "POST":
        if "file" in request.files:
            file = request.files["file"]
            if file and file.filename.endswith(".pdf"):
                filepath = os.path.join(app.config['UPLOAD_FOLDER'], file.filename)
                file.save(filepath)
                extracted_text = extract_text_from_pdf(filepath)
                response = "✅ File uploaded and processed successfully!"
        elif "query" in request.form:
            user_query = request.form.get("query")
            response = simple_chatbot(user_query)

    return render_template_string('''
<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <title>Financial Chatbot + Statement Analyzer</title>
    <style>
        body { font-family: 'Segoe UI', sans-serif; margin: 0; padding: 0; background-color: #f4f6f8; }
        .container { max-width: 700px; margin: 40px auto; padding: 20px; background-color: #ffffff;
                     border-radius: 12px; box-shadow: 0px 4px 15px rgba(0,0,0,0.1); }
        h1 { text-align: center; color: #2c3e50; }
        form { margin-top: 20px; }
        input[type="text"], input[type="file"] {
            width: 70%; padding: 10px; border-radius: 8px; border: 1px solid #ccc; font-size: 16px;
        }
        input[type="submit"] {
            padding: 10px 16px; font-size: 16px; border: none; border-radius: 8px;
            background-color: #3498db; color: white; cursor: pointer;
        }
        input[type="submit"]:hover { background-color: #2980b9; }
        .response-box {
            background-color: #ecf0f1; padding: 15px; border-radius: 10px;
            margin-top: 20px; white-space: pre-line;
        }
        .button-row { margin: 10px 0; display: flex; flex-wrap: wrap; gap: 10px; }
        .quick-button {
            padding: 8px 12px; border: none; border-radius: 6px;
            background-color: #bdc3c7; cursor: pointer; font-size: 14px;
        }
        .quick-button:hover { background-color: #95a5a6; }
    </style>
</head>
<body>
    <div class="container">
        <h1>📊 Financial Statement Analyzer Chatbot</h1>

        <form method="POST" enctype="multipart/form-data">
            <label><strong>Upload a 10-K / Financial PDF:</strong></label><br><br>
            <input type="file" name="file">
            <input type="submit" value="Upload">
        </form>

        <form method="POST">
            <label><strong>Ask a financial question:</strong></label><br><br>
            <input type="text" name="query" placeholder="e.g. What's the total revenue?">
            <input type="submit" value="Ask">
        </form>

        <div class="button-row">
            <form method="POST"><input type="hidden" name="query" value="What is the total revenue?"><button class="quick-button">Total Revenue</button></form>
            <form method="POST"><input type="hidden" name="query" value="How did net income change from 2022 to 2023?"><button class="quick-button">Net Income</button></form>
            <form method="POST"><input type="hidden" name="query" value="What is in the uploaded statement?"><button class="quick-button">Analyze Uploaded Statement</button></form>
            <form method="POST"><input type="hidden" name="query" value="Show me a revenue chart"><button class="quick-button">📊 Revenue Chart</button></form>
        </div>

        {% if response %}
            <div class="response-box">{{ response | safe }}</div>
        {% endif %}
    </div>
</body>
</html>
''', response=response)

# Start ngrok tunnel
public_url = ngrok.connect(5000)
print(" * Public URL:", public_url)

# Run the Flask app
app.run(port=5000)


 * Public URL: NgrokTunnel: "https://679d-34-28-119-236.ngrok-free.app" -> "http://localhost:5000"
 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit
INFO:werkzeug:127.0.0.1 - - [01/Jul/2025 23:52:48] "GET / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [01/Jul/2025 23:52:48] "GET /favicon.ico HTTP/1.1" 404 -
INFO:werkzeug:127.0.0.1 - - [01/Jul/2025 23:52:54] "POST / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [01/Jul/2025 23:53:25] "POST / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [01/Jul/2025 23:53:43] "POST / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [01/Jul/2025 23:53:49] "POST / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [01/Jul/2025 23:54:33] "POST / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [01/Jul/2025 23:55:54] "POST / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [01/Jul/2025 23:56:07] "POST / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [01/Jul/2025 23:56:19] "POST / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [01/Jul/2